## Analyze NaN's 

In this Notebook we study the presence of NAN in each stock dataset, we want to know the following information:

1-Numbre of NAN's

2-Presence of holes which mean how many we have of consecutive days of inactivity  .

3-the length of holes

4-the start and end date of each hole


In [1]:
import pandas as pd 
import glob
from os import listdir
from os.path import isfile, join
import numpy as np
import itertools
import datetime as dt

In [2]:
PATH = r'C:\Users\Zakaria Bouchegoura\Desktop\Nouveau dossier (5)\archive\stock_market_data\nasdaq\csv'

### Loading All Stock Market

In [3]:
# getting path's of all datasets
dataset_files = glob.glob(PATH + "/*.csv") 


datasets = {} # dictionary to save each stock-market dataset by its Index

for filename in dataset_files:
    stock_name=filename.split("\\")[-1][:-4] #getting the Index of the stock
    df = pd.read_csv(filename, index_col=None, header=0) 
    datasets[stock_name]=df 

In [4]:
# adding to the global environment, global variables of each stock market called by its stock index
locals().update(datasets) 

In [5]:
#Example: if you want any to view the dataset of Apple you need just to type it indice "AAPL"
AAPL # Hooray !

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,12-12-1980,0.128348,0.128348,469033600,0.128906,0.128348,0.100178
1,15-12-1980,0.121652,0.122210,175884800,0.122210,0.121652,0.094952
2,16-12-1980,0.112723,0.113281,105728000,0.113281,0.112723,0.087983
3,17-12-1980,0.115513,0.115513,86441600,0.116071,0.115513,0.090160
4,18-12-1980,0.118862,0.118862,73449600,0.119420,0.118862,0.092774
...,...,...,...,...,...,...,...
10454,31-05-2022,146.839996,149.070007,103718400,150.660004,148.839996,148.839996
10455,01-06-2022,147.679993,149.899994,74286600,151.740005,148.710007,148.710007
10456,02-06-2022,146.860001,147.830002,72348100,151.270004,151.210007,151.210007
10457,03-06-2022,144.460007,146.899994,88471400,147.970001,145.380005,145.380005


## Analyse the NaN

At the start, we need to create some functions that will help us to make a clear resume of our NaN, we are interested in counting the number of holes in each dataset and localizing the starting and ending date of holes in the dataset of each stock.

In [7]:
def stock_nan(STOCK):   
    count=STOCK['Close'].isnull().sum()
    return count   

In [35]:
NBR_NAN=[]
FRQ_NAN=[]
for stock in datasets.keys() :
    nan=stock_nan(datasets[stock])
    frq=(nan/datasets[stock].shape[0])*100
    NBR_NAN.append(nan)
    FRQ_NAN.append(frq)

nbr_nan = pd.DataFrame(list(zip(list(datasets.keys()), NBR_NAN, FRQ_NAN)),
               columns =['Stock', 'nombre_NAN',"Frequency_NAN"])

A First Summary of NaN's of our dataset

In [36]:
nbr_nan # a data frame resuming the numbre and frequency of each stock

,Stock,nombre_NAN,Frequency_NAN
0,AAL,0,0.000000
1,AAME,0,0.000000
2,AAOI,0,0.000000
3,AAON,0,0.000000
4,AAPL,0,0.000000
...,...,...,...
1604,ZEUS,0,0.000000
1605,ZION,0,0.000000
1606,ZNGA,8,0.303836
1607,ZOES,890,43.351193


In [37]:
liste_stock_nan=nbr_nan.loc[nbr_nan["nombre_NAN"]!=0].Stock.tolist() # getting the name of stocks who contain NaN

In [47]:
datasets_nan={} # create another datasets with stock that contains NaN

for stock in liste_stock_nan :
    datasets_nan[stock]=datasets[stock]

#### Exploring the holes of each stock market dataset

In [56]:
def stock_hole_reduc(STOCK):

    """

    this function provide the starting and ending date of each hole in dataset
    
    """

    local=pd.DataFrame(np.argwhere(np.isnan(np.array(STOCK.Close))),columns=["indic_seq"])
    local["index"]=local.indic_seq
    STOCK["index"]=STOCK.index
    local2=local.merge(STOCK, on="index", how = 'inner')
    local2["indic"]=local2["index"].diff().ne(1).cumsum()
    local3=local2.groupby('indic').apply(lambda x: x.iloc[[0, -1]])
    local3.drop(["Close","Open","Low","Volume","High","Adjusted Close","index","indic","indic_seq"],axis=1,inplace=True)

    return local3

In [57]:
stock_hole_reduc(ABCD) # Example on the stock "ABCD"

Date
indic                
1     0    22-09-2015
      0    22-09-2015
2     1    26-12-2018
      1    26-12-2018
3     2    28-12-2018
      867  06-06-2022

In [58]:
def hole(STOCK):

    """

    this function provide the len of each hole in the dataset. Notice : a dataset can contain many holes.
    
    """
    local = pd.Series(STOCK.Close)
    len_holes = [len(list(g)) for k, g in itertools.groupby(local, lambda x: np.isnan(x)) if k]
    return len_holes

In [59]:
hole(ABCD) # example with same stock "ABCD"

[1, 1, 866]

#### Assembling the tools to make our summary of NaN :

We use "datasets_nan" and the two functions "stock_hole_reduc" and "hole" to create "Analyze_nan"


In [60]:

STK_NAN=[] # Contain the lenght of each hole by stock
nbr_nan=[] # Contain the total number of NaN on each stock
freq_nan=[] # the frequency of the NaN
last_ele=[] # showing the last hole of each stack, this inofrmation can be usefull to know when it was the last activity of the stock
nbr_holes=[] # a dataset can contain more than one hole
last_nan_begin=[] # the starting date of the last inactivity 
last_nan_end=[] # the ending date of the last inactivity


for stock in datasets_nan.keys() :
    nan_date=stock_hole_reduc(datasets_nan[stock])
    nan=hole(datasets_nan[stock])
    STK_NAN.append(nan)
    last_ele.append(nan[-1])
    nbr_nan.append(sum(nan))
    freq_nan.append(sum(nan)/len(datasets_nan[stock]))
    nbr_holes.append(len(nan))
    last_nan_begin.append(nan_date.Date.iloc[-2])
    last_nan_end.append(nan_date.Date.iloc[-1])

In [61]:
#saving the previous results on a DataFrame
Analyze_nan = pd.DataFrame(list(zip(liste_stock_nan,nbr_nan,freq_nan,nbr_holes,STK_NAN,last_ele,last_nan_begin,last_nan_end)),
               columns =['Stock','nbr_nan','frequency_nan','nbr_holes','holes','last_ele','begin','end'])

In [62]:
Analyze_nan

,Stock,nbr_nan,frequency_nan,nbr_holes,holes,last_ele,begin,end
0,AAVL,777,0.672727,2,"[37, 740]",740,01-07-2019,06-06-2022
1,ABCD,868,0.276081,3,"[1, 1, 866]",866,28-12-2018,06-06-2022
2,ABCO,559,0.222798,483,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",66,03-03-2022,06-06-2022
3,ACFC,1030,0.231513,16,"[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1014,29-05-2018,06-06-2022
4,ACTA,1040,0.180964,1,[1040],1040,20-04-2018,06-06-2022
...,...,...,...,...,...,...,...,...
142,XENT,14,0.007067,1,[14],14,16-05-2022,03-06-2022
143,XPLR,1166,0.185669,153,"[1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 2, 2, 1, 1, ...",960,14-08-2018,06-06-2022
144,ZAZA,905,0.484216,15,"[3, 17, 1, 1, 25, 3, 1, 1, 1, 1, 1, 1, 1, 1, 847]",847,25-01-2019,03-06-2022
145,ZNGA,8,0.003038,1,[8],8,24-05-2022,03-06-2022


#### A Complete dataset analysis 


The below function gives a detailed analysis of each hole and its start and end date, it will be useful if you are studying a specific stock market action.


In [63]:
def stock_hole(STOCK):


    local=pd.DataFrame(np.argwhere(np.isnan(np.array(STOCK.Close))),columns=["indic_seq"])
    local["index"]=local.indic_seq
    STOCK["index"]=STOCK.index
    local2=local.merge(STOCK, on="index", how = 'inner')
    local2["indic"]=local2["index"].diff().ne(1).cumsum()
    local4 = pd.Series(STOCK.Close)
    len_holes = [len(list(g)) for k, g in itertools.groupby(local4, lambda x: np.isnan(x)) if k]
    local5=STOCK['Close'].isnull().sum()
    local6=local5/STOCK.shape[0]
    local3=local2.groupby('indic').apply(lambda x: x.iloc[[0, -1]])
    local3.drop(["Close","Open","Low","Volume","High","Adjusted Close","index","indic","indic_seq"],axis=1,inplace=True)
    
    print("Numbre of NaN in the stock",local5)
    print("Frequency of NaN in stock",local6)
    print("Numbre of Nan Holes",len(len_holes))
    print("lenght each of Nan Holes",len_holes)
    print("Star-End of each hole",local3)


    return

In [64]:
stock_hole(ABCD) # always with example "ABCD" stock :)

Numbre of NaN in the stock 868
Frequency of NaN in stock 0.27608142493638677
Numbre of Nan Holes 3
lenght each of Nan Holes [1, 1, 866]
Star-End of each hole                  Date
indic                
1     0    22-09-2015
      0    22-09-2015
2     1    26-12-2018
      1    26-12-2018
3     2    28-12-2018
      867  06-06-2022
